In [3]:
import torch
import torch.nn.functional as F

def retrieve_similar_samples(h_q, M, k):
    """
    根据查询向量h_q从记忆M中检索最相似的前k个样本。
    M是形状为(num_samples, d+2)的张量，d是h_i的维度。
    """
    # 计算相似度（这里使用点积）
    similarities = torch.matmul(M[:, :d], h_q.unsqueeze(-1)).squeeze(-1)
    # 获取相似度最高的前k个样本的索引
    topk_indices = torch.topk(similarities, k).indices
    return M[topk_indices]

def compute_attention_weights(similar_samples, tau):
    """
    计算相似样本的注意力权重。
    """
    # 使用softmax计算注意力权重
    attention_weights = F.softmax(similar_samples / tau, dim=0)
    return attention_weights

def corrected_model_prediction(h_q, M, k, tau, gamma, lambda_val):
    """
    使用错误记忆来计算校正后的模型预测。
    """
    similar_samples = retrieve_similar_samples(h_q, M, k)
    
    # 计算注意力权重
    similarities = torch.matmul(similar_samples[:, :d], h_q.unsqueeze(-1)).squeeze(-1)
    attention_weights = compute_attention_weights(similarities, tau)
    
    # 计算加权的真实值和基础模型预测值
    y_bar = torch.sum(attention_weights * similar_samples[:, d])
    y_base_bar = torch.sum(attention_weights * similar_samples[:, d+1])
    
    # 校正后的预测
    y_pred = (1 - lambda_val) * y_base_bar + lambda_val * y_bar
    
    return y_pred

# 示例参数
d = 128  # 隐藏向量的维度
num_samples = 1000  # 记忆中的样本数量
k = 10  # 要检索的样本数量
tau = 0.1  # 温度参数
gamma = 1  # 误差测量的权重
lambda_val = 0.5  # 校正权重

# 模拟一个记忆M，包含随机向量h_i, y_i和y_base_i
M = torch.randn(num_samples, d + 2)  # 每行：[h_i, y_i, y_base_i]

# 模拟一个查询隐藏向量h_q
h_q = torch.randn(d)

# 获取校正后的模型预测
y_pred = corrected_model_prediction(h_q, M, k, tau, gamma, lambda_val)
print(y_pred)


tensor(0.6704)


In [ ]:
def window_print()
    

# window_print('你好')